In [1]:
import pandas as pd
from pyproj import Proj, transform
import fiona
from fiona.crs import from_epsg
import logging
import sys


# outProj = Proj(init='epsg:3857')
# inProj = Proj(init='epsg:4326') # It is equal to WSG

In [8]:
shape = fiona.open("Mapas_sim/env/amb_app_lagoas_25k_inea.shp")
print (shape.schema)


{'properties': OrderedDict([('id', 'float:19'), ('Classes', 'str:254'), ('Perimetro_', 'float:33.15'), ('Area_Hecta', 'float:33.15')]), 'geometry': 'Polygon'}


In [11]:
for i in shape:
    print(i['id'])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17


In [13]:
print(shape[6]['geometry']['type'])

Polygon


In [14]:
# Transformar escala lag, long em Metros no globo achatado

# salvando o esquema e pegando a projecao de entrada
sink_schema = shape.schema.copy()
p_in = Proj(shape.crs)

# definindo um novo shape de saida, no mesmo local do shape atual com a projeção mudada
with fiona.open(
        'lagoa_metros.shp', 'w',
        crs=from_epsg(3857),
        driver=shape.driver,
        schema=sink_schema,
        ) as sink:

    p_out = Proj(sink.crs)

    # fazendo apenas para o Chico Mendes (esse arquivo tem Polygon e MultiPolygon)
    # nesse caso, se fizesse para o arquivo todo, nao seria contemplado alguns shapes
    f = shape[6]

    try:
        # verifica se eh da estrutura poligono
        assert f['geometry']['type'] == "Polygon"
        new_coords = []
        # para todas as coordenadas do poligono
        for ring in f['geometry']['coordinates']:
            x2, y2 = transform(p_in, p_out, *zip(*ring)) #transforma
            new_coords.append(zip(x2, y2)) # salva a nova lista de coordenadas
        f['geometry']['coordinates'] = new_coords
        sink.write(f)

    except Exception:
        # Writing uncleanable features to a different shapefile
        # is another option.
        logging.exception("Error transforming feature %s:", f['id'])

/opt/anaconda/lib/python3.7/site-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))
/opt/anaconda/lib/python3.7/site-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))
